In [ ]:
import kagglehub
import shutil
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import os
import pandas as pd

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rtlmhjbn/ip02-dataset")

print("Path to dataset files:", path)

In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset for IP102
class IP102Dataset(Dataset):
    def __init__(self, data_file, root_dir, transform=None):
        self.data = pd.read_csv(data_file, sep=' ', header=None, names=['image', 'label'])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.data.iloc[idx, 1]), self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.data.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Load class names from classes.txt
def load_class_names(classes_file):
    try:
        with open(classes_file, 'r') as f:
            classes = [line.strip().split(' ', 1)[1] for line in f.readlines()]
        return classes
    except Exception as e:
        print(f"Error loading classes.txt: {str(e)}")
        return None

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_acc = 100 * val_correct / val_total
        print(f"Validation Acc: {val_acc:.2f}%")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'resnet50_ip102.pth')

# Testing function
def test_model(model, test_loader, class_names):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_acc = 100 * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")

def main():
    # Paths (update these based on your setup)
    data_dir = "/root/.cache/kagglehub/datasets/rtlmhjbn/ip02-dataset/versions/1/classification"  # Update with path to classification directory
    classes_file = "/root/.cache/kagglehub/datasets/rtlmhjbn/ip02-dataset/versions/1/classes.txt"  # Update with path to classes.txt
    train_file = "/root/.cache/kagglehub/datasets/rtlmhjbn/ip02-dataset/versions/1/train.txt"
    test_file = "/root/.cache/kagglehub/datasets/rtlmhjbn/ip02-dataset/versions/1/test.txt"
    valid_file = "/root/.cache/kagglehub/datasets/rtlmhjbn/ip02-dataset/versions/1/val.txt"

    # Validate paths
    for path in [data_dir, classes_file, train_file, test_file, valid_file]:
        if not os.path.exists(path):
            print(f"Error: {path} does not exist.")
            return

    # Load class names
    class_names = load_class_names(classes_file)
    if class_names is None:
        return
    num_classes = len(class_names)

    # Create datasets
    train_dataset = IP102Dataset(train_file, os.path.join(data_dir, 'train'), transform=train_transform)
    val_dataset = IP102Dataset(valid_file, os.path.join(data_dir, 'val'), transform=val_test_transform)
    test_dataset = IP102Dataset(test_file, os.path.join(data_dir, 'test'), transform=val_test_transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    # Initialize model
    model = models.resnet50(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

    # Train and validate
    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=1)

    # Test
    test_model(model, test_loader, class_names)

main()